Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [7]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!mkdir drive
!google-drive-ocamlfuse drive

In [6]:
!ls drive/ColabUtils/udacity

1_notmnist.ipynb	4_convolutions.ipynb  Dockerfile
2_fullyconnected.ipynb	5_word2vec.ipynb      notMNIST.pickle
3_regularization.ipynb	6_lstm.ipynb	      README.md


In [8]:
pickle_file = 'drive/ColabUtils/udacity/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [9]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [7]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



Let's run this computation and iterate:

In [8]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 15.653460
Training accuracy: 10.9%
Validation accuracy: 13.1%
Loss at step 100: 2.321112
Training accuracy: 71.2%
Validation accuracy: 71.1%
Loss at step 200: 1.869491
Training accuracy: 74.2%
Validation accuracy: 73.7%
Loss at step 300: 1.624902
Training accuracy: 75.7%
Validation accuracy: 74.7%
Loss at step 400: 1.456742
Training accuracy: 76.8%
Validation accuracy: 75.2%
Loss at step 500: 1.329738
Training accuracy: 77.3%
Validation accuracy: 75.7%
Loss at step 600: 1.229237
Training accuracy: 78.0%
Validation accuracy: 75.9%
Loss at step 700: 1.147428
Training accuracy: 78.7%
Validation accuracy: 76.2%
Loss at step 800: 1.079410
Training accuracy: 79.2%
Validation accuracy: 76.2%
Test accuracy: 82.8%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [0]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [0]:
for step in range(10):
  print ((step * batch_size) % (train_labels.shape[0] - batch_size))
train_labels.shape, batch_size, train_labels.shape[0] - batch_size

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 13.456728
Minibatch accuracy: 6.2%
Validation accuracy: 16.5%
Minibatch loss at step 1000: 1.153790
Minibatch accuracy: 78.9%
Validation accuracy: 77.1%
Minibatch loss at step 2000: 0.966280
Minibatch accuracy: 78.1%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 0.735428
Minibatch accuracy: 85.2%
Validation accuracy: 78.9%
Test accuracy: 86.4%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch.nn as nn
from torch.autograd import Variable as V

In [0]:
class LogisticRegression(nn.Module):
  def __init__(self, input_size, hid, num_classes):
    super(LogisticRegression, self).__init__()
    self.l1 = nn.Linear(input_size, hid)
    self.l2 = nn.Linear(hid, num_classes)
    self.relu = nn.ReLU()
#     self.softmax = nn.Softmax()
  def forward(self, x):
    relu = self.relu(self.l1(x))
    out = self.l2(relu)
    return out

In [0]:
model = LogisticRegression(image_size * image_size, 1024, num_labels).float()
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=.5)

In [21]:
train_labels_ind = np.array([ np.where(r==1)[0][0] for r in train_labels ])
train_labels_ind

array([4, 9, 6, ..., 2, 4, 4])

In [0]:
def onehot2ind(x): return np.array([ np.where(r==1)[0][0] for r in x ], dtype=np.long)

In [22]:
train_labels_ind.shape

(200000,)

In [24]:
for step in range(num_steps):
  offstep = (step * batch_size) % (train_labels.shape[0] - batch_size)
  batch_data = train_dataset[offset:(offset + batch_size), :]
  batch_labels = train_labels_ind[offset:(offset + batch_size)]
  images = V(torch.Tensor(batch_data))
  labels = V(torch.Tensor(batch_labels)).long()

  # Forward + Backward + Optimize
  optimizer.zero_grad()
  outputs = model(images)
  loss = criterion(outputs, labels)
  loss_val = criterion()
  loss.backward()
  optimizer.step()

  if (step) % 100 == 0:
      print ('step: %d, Loss: %.4f' 
             % (step, loss.data[0]))


step: 0, Loss: 2.3111
step: 100, Loss: 0.0076
step: 200, Loss: 0.0031
step: 300, Loss: 0.0019
step: 400, Loss: 0.0013
step: 500, Loss: 0.0010
step: 600, Loss: 0.0008
step: 700, Loss: 0.0007
step: 800, Loss: 0.0006
step: 900, Loss: 0.0005
step: 1000, Loss: 0.0004
step: 1100, Loss: 0.0004
step: 1200, Loss: 0.0004
step: 1300, Loss: 0.0003
step: 1400, Loss: 0.0003
step: 1500, Loss: 0.0003
step: 1600, Loss: 0.0003
step: 1700, Loss: 0.0002
step: 1800, Loss: 0.0002
step: 1900, Loss: 0.0002
step: 2000, Loss: 0.0002
step: 2100, Loss: 0.0002
step: 2200, Loss: 0.0002
step: 2300, Loss: 0.0002
step: 2400, Loss: 0.0002
step: 2500, Loss: 0.0002
step: 2600, Loss: 0.0002
step: 2700, Loss: 0.0001
step: 2800, Loss: 0.0001
step: 2900, Loss: 0.0001
step: 3000, Loss: 0.0001


In [25]:
valid_dataset.shape, valid_labels.shape

((10000, 784), (10000, 10))

In [31]:
valid_labels_ind = onehot2ind(valid_labels)
valid_labels_ind.shape

(10000,)

In [34]:
valid_labels_ind.size

10000

In [45]:
correct = 0
total = 0
for step in range(len(valid_labels_ind)):
    images = V(torch.Tensor(valid_dataset))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += valid_labels_ind.size
    correct += (predicted == torch.Tensor(valid_labels_ind).long()).sum()
      print('Accuracy of the model on the [%d] test images: %d %%' % (total, (100 * correct / total)))

Accuracy of the model on the [630000] test images: 73 %
Accuracy of the model on the [640000] test images: 73 %
Accuracy of the model on the [650000] test images: 73 %
Accuracy of the model on the [660000] test images: 73 %
Accuracy of the model on the [670000] test images: 73 %
Accuracy of the model on the [680000] test images: 73 %


KeyboardInterrupt: ignored

In [0]:
for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

---
#Problem
Same with tf

---

In [49]:
num_labels

10

In [50]:
tf_train_dataset

<tf.Tensor 'Const:0' shape=(10000, 784) dtype=float32>

In [10]:
train_subset = 10000

graph = tf.Graph()

num_hidden = 1024

def predict(x, w1, b1, w2, b2):
  lin1 = tf.matmul(x, w1) + b1
  lin2 = tf.matmul(tf.nn.relu(lin1), w2) + b2
  return lin2

with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
 
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  w1 = tf.Variable(
      tf.truncated_normal([image_size * image_size, num_hidden]))
  b1 = tf.Variable(tf.zeros([num_hidden]))
  
  w2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = predict(tf_train_dataset, w1, b1, w2, b2)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    predict(tf_valid_dataset, w1, b1, w2, b2))
  test_prediction = tf.nn.softmax(predict(tf_test_dataset, w1, b1, w2, b2))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [0]:
num_steps = 801

In [0]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [15]:
with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 295.685791
Training accuracy: 11.3%
Validation accuracy: 35.6%
Loss at step 100: 2.901247
Training accuracy: 93.0%
Validation accuracy: 79.7%
Loss at step 200: 0.298337
Training accuracy: 98.4%
Validation accuracy: 79.5%
Loss at step 300: 0.016677
Training accuracy: 99.8%
Validation accuracy: 79.2%
Loss at step 400: 0.010276
Training accuracy: 99.8%
Validation accuracy: 79.7%
Loss at step 500: 0.008914
Training accuracy: 99.9%
Validation accuracy: 79.8%
Loss at step 600: 0.009823
Training accuracy: 99.9%
Validation accuracy: 79.8%
Loss at step 700: 0.008193
Training accuracy: 99.9%
Validation accuracy: 79.8%
Loss at step 800: 0.022379
Training accuracy: 99.9%
Validation accuracy: 79.9%
Test accuracy: 87.0%
